In [1]:
from pyspark.sql import SparkSession

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1687204372890_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
spark = SparkSession.builder.getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from pyspark.sql.types import StructType, StructField, StringType,IntegerType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
schema = StructType([
 StructField("year", IntegerType()), 
 StructField("month", StringType()), 
 StructField("day", IntegerType()),  
 StructField("weekday", StringType())  ,
 StructField("hour", IntegerType())  ,
 StructField("atm_status", StringType()) ,
 StructField("atm_id", StringType()) ,
 StructField("atm_manufacturer", StringType()) ,
 StructField("atm_location", StringType()) ,
 StructField("atm_streetname", StringType()) ,
 StructField("atm_street_number", IntegerType()) ,
 StructField("atm_zipcode", IntegerType())  ,
 StructField("atm_lat", StringType())  ,
 StructField("atm_lon", StringType())  ,
 StructField("currency", StringType()) ,
 StructField("card_type", StringType())  ,
 StructField("transaction_amount", IntegerType()),
 StructField("service", StringType())  ,
 StructField("message_code", StringType()) ,
 StructField("message_text", StringType()) ,
 StructField("weather_lat", StringType())  ,
 StructField("weather_lon", StringType())  ,
 StructField("weather_city_id", IntegerType()) ,
 StructField("weather_city_name", StringType())  ,
 StructField("temp", StringType()) ,
 StructField("pressure", IntegerType())  ,
 StructField("humidity", IntegerType())  ,
 StructField("wind_speed", IntegerType())  ,
 StructField("wind_deg", IntegerType())  ,
 StructField("rain_3h", StringType())  ,
 StructField("clouds_all", IntegerType())  ,
 StructField("weather_id", IntegerType())  ,
 StructField("weather_main", StringType()) ,
 StructField("weather_description", StringType())
 
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Reading data from Hadoop

In [9]:
denormalized_df = spark.read.schema(schema).parquet("/user/root/assignment//a6d88055-0b19-4572-8e63-ce01866a70be.parquet")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Cleaning Column Names

In [10]:
denormalized_df=denormalized_df.withColumnRenamed("atm_location", "location") \
                                    .withColumnRenamed("atm_streetname", "streetname") \
                                    .withColumnRenamed("atm_street_number", "street_number")  \
                                    .withColumnRenamed("atm_zipcode", "zipcode") \
                                    .withColumnRenamed("atm_lat", "lat") \
                                    .withColumnRenamed("atm_lon", "lon") \
                                    .withColumnRenamed("atm_id", "atm_number")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.sql.functions import concat, lit, to_date
from pyspark.sql.functions import monotonically_increasing_id

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Preparing Data for Date Dimention

In [12]:
# Add an ID column to the denormalized DataFrame
#denormalized_df_with_id = denormalized_df.withColumn("date_id", monotonically_increasing_id())

# Select relevant columns for the fact dimension

fact_dim_df = denormalized_df.select("year", "month","day","hour","weekday").distinct()

df_with_date = fact_dim_df.withColumn("full_date_time", to_date(concat(fact_dim_df["Year"], lit("-"), fact_dim_df["Month"], lit("-"), fact_dim_df["Day"], lit(" "), fact_dim_df["hour"]), "yyyy-MM-dd HH"))\
                          .withColumn("date_id", monotonically_increasing_id())

df_with_date.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

#### Preparing Data for Card Type Dimention

In [13]:

card_type_dim = denormalized_df.select("card_type").distinct()

card_type_dim_with_id = card_type_dim.withColumn("card_type_id", monotonically_increasing_id())

card_type_dim_with_id.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(card_type='Visa Dankort - on-us', card_type_id=8589934592)

#### Preparing Data for Location Dimention

In [14]:
location_dim = denormalized_df.select("location","streetname","street_number","zipcode","lat","lon").distinct()

location_dim_with_id = location_dim.withColumn("location_id", monotonically_increasing_id())


location_dim_with_id.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

#### Preparing Data for ATM Dimention

In [15]:
DIM_ATM = denormalized_df.select("location","streetname","street_number","zipcode","lat","lon","atm_number","atm_manufacturer").distinct()

joined_df = DIM_ATM.join(location_dim_with_id, on=["location", "streetname","street_number","zipcode","lat","lon"], how="left")

result_df = joined_df.select("atm_number","atm_manufacturer","location_id").withColumn("atm_id", monotonically_increasing_id()).withColumnRenamed("location_id", "atm_location_id")

result_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

#### Preparing Fact Table data

In [16]:
denormalized_df_with_id=denormalized_df.withColumn("trans_id", monotonically_increasing_id())
join_location=denormalized_df_with_id.join(location_dim_with_id,on=["location", "streetname","street_number","zipcode","lat","lon"], how="left")
join_atm=join_location.join(result_df,((join_location["location_id"]==result_df["atm_location_id"]) & (join_location["atm_number"]==result_df["atm_number"]) &(join_location["atm_manufacturer"]==result_df["atm_manufacturer"]) ),how="left")
join_date=join_atm.join(df_with_date,on=["year", "month","day","hour","weekday"], how="left")
join_card=join_date.join(card_type_dim_with_id,on=["card_type"], how="left")
FACT_ATM_TRANS=join_card.select("trans_id","atm_id","location_id","date_id","card_type_id","atm_status","currency","service","transaction_amount","message_code","message_text","rain_3h","clouds_all","weather_id","weather_main","weather_description").withColumnRenamed("location_id", "weather_loc_id")
FACT_ATM_TRANS.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

#### Loading all the fact and Dimention tables to S3

In [17]:
s3_output_path = "s3://puniths3redshift/All_tables/FACT_ATM_TRANS"
FACT_ATM_TRANS.write.csv(s3_output_path,quote='"', escape='"')
s3_output_path = "s3://puniths3redshift/All_tables/DATE_DIM"
df_with_date.write.csv(s3_output_path,quote='"', escape='"')
s3_output_path = "s3://puniths3redshift/All_tables/CARD_TYPE_DIM"
card_type_dim_with_id.write.csv(s3_output_path,quote='"', escape='"')
s3_output_path = "s3://puniths3redshift/All_tables/LOCATION_DIM"
location_dim_with_id.write.csv(s3_output_path,quote='"', escape='"')
s3_output_path = "s3://puniths3redshift/All_tables/ATM_DIM"
result_df.write.csv(s3_output_path,quote='"', escape='"')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…